Clumper可以用来处理嵌套样式的json数据结构。


## 安装

In [6]:
!pip3 install clumper

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


为了展示Clumper如何工作，我准备了pokemon.json， 由列表组成(该列表由多个字典组成)，下面是pokemon.json部分内容

In [7]:
import json

with open('pokemon.json') as jsonf:
    pokemon = json.loads(jsonf.read())
    
pokemon[:2]

[{'name': 'Bulbasaur',
  'type': ['Grass', 'Poison'],
  'total': 318,
  'hp': 45,
  'attack': 49},
 {'name': 'Ivysaur',
  'type': ['Grass', 'Poison'],
  'total': 405,
  'hp': 60,
  'attack': 62}]

我们准备的pokemon.json列表中大概有800个字典，数量级刚刚好，不会因为太大导致电脑无法运行数据分析，也不会太小导致手动操作性价比更高。

## Example
### 基本操作

In [19]:
from clumper import Clumper

list_of_dicts = [
    {'a': 7, 'b': 2},
    {'a': 2, 'b': 4},
    {'a': 3, 'b': 6}
]

(Clumper(list_of_dicts)
 .mutate(c=lambda d: d['a']+d['b'])
 .sort(lambda d: d['c'])
 .collect()
)

[{'a': 2, 'b': 4, 'c': 6}, {'a': 7, 'b': 2, 'c': 9}, {'a': 3, 'b': 6, 'c': 9}]

### 代码解析
**Step1**

首先使用**mutate**方法，该方法可以在每条记录中生成新变量。
![](img/first-mutate.png)
结算结果仍为Clumper类


**Step2**

接下来对mutate之后的数据进行排序
![](img/then-sort.png)
得到的结果仍为Clumper类。

从上面的小代码案例中，可以看到整套流程像是一个流水线车间，每一行就是一个生成环节，生产环节之间使用.连接起来。

In [20]:
from clumper import Clumper
    
(Clumper(pokemon)
 .keep(lambda d: len(d['type'])==1)  #保留type长度为1的字典
 .mutate(type=lambda d: d['type'][0], #type值从列表变为字符串
         ratio=lambda d: d['attack']/d['hp']) #新建ratio
 .select('name', 'type', 'ratio')   #字典最后只保留name， type， ratio三个字段
 .sort(lambda d: d['ratio'], reverse=True) #按照ratio降序排列
 .head(5) #只保留前5个
 .collect() #转成列表显示
)

[{'name': 'Diglett', 'type': 'Ground', 'ratio': 5.5},
 {'name': 'DeoxysAttack Forme', 'type': 'Psychic', 'ratio': 3.6},
 {'name': 'Krabby', 'type': 'Water', 'ratio': 3.5},
 {'name': 'DeoxysNormal Forme', 'type': 'Psychic', 'ratio': 3.0},
 {'name': 'BanetteMega Banette', 'type': 'Ghost', 'ratio': 2.578125}]